## Проект по SQL. Описание.

Компания решила купилть крупный сервис для чтения книг по подписке. Нам необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

`Таблица books` cодержит данные о книгах:
-	book_id — идентификатор книги;
-	author_id — идентификатор автора;
-	title — название книги;
-	num_pages — количество страниц;
-	publication_date — дата публикации книги;
-	publisher_id — идентификатор издателя.

`Таблица authors` cодержит данные об авторах:
-	author_id — идентификатор автора;
-	author — имя автора.

`Таблица publishers` cодержит данные об издательствах:
-	publisher_id — идентификатор издательства;
-	publisher — название издательства;

`Таблица ratings` cодержит данные о пользовательских оценках книг:
-	rating_id — идентификатор оценки;
-	book_id — идентификатор книги;
-	username — имя пользователя, оставившего оценку;
-	rating — оценка книги.

`Таблица reviews` cодержит данные о пользовательских обзорах на книги:
-	review_id — идентификатор обзора;
-	book_id — идентификатор книги;
-	username — имя пользователя, написавшего обзор;
-	text — текст обзора.


## Задачи.
1. Выяснить, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем из анализа брошюры);
4. Определить автора с самой высокой средней оценкой книг, необходимо учитывать только книги с 50 и более оценками;
5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.


## Подключение к базе данных

In [1]:
# импорт библиотек
import pandas as pd
from sqlalchemy import create_engine

# # отключим предупреждения Anaconda
import warnings
warnings.simplefilter('ignore')


pd.set_option('display.float_format', '{:,.2f}'.format) # вывод всех значений в столбцах с двумя знаками после запятой
pd.set_option('display.max_columns', None) # вывод всех столбцов без скрытых за многоточием
pd.options.display.max_colwidth = 150 # вывод полного текста в ячейке

In [2]:
# установка параметров
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', # адрес сервера
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])  
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

## Исследование

### Взгляд на данные. Первые строки таблиц

In [4]:
# обращение к базе данных через строку query, которая отбирает все*столбцы из каждой таблицы

query = ''' SELECT *
         FROM books
        '''

query1 = ''' SELECT *
         FROM authors
        '''

query2 = ''' SELECT *
         FROM publishers
        '''

query3 = ''' SELECT *
         FROM ratings
        '''
query4 = ''' SELECT *
         FROM reviews
        '''

In [5]:
# вывод таблиц в питоне
books = pd.io.sql.read_sql(query, con = engine) 
authors = pd.io.sql.read_sql(query1, con = engine) 
publishers = pd.io.sql.read_sql(query2, con = engine) 
ratings = pd.io.sql.read_sql(query3, con = engine) 
reviews = pd.io.sql.read_sql(query4, con = engine) 

In [6]:
# функция для вывода общей информации о датасете
def basic_func(data):
        display(data.head())
        display(data.info())
        display(data.describe().T)
        print('Количество явных дуликатов в таблице', data.duplicated().sum())
        print('Размер датасета:', data.shape)

In [7]:
basic_func(books)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


None

,count,mean,std,min,25%,50%,75%,max
book_id,"1,000.00",500.50,288.82,1.00,250.75,500.50,750.25,"1,000.00"
author_id,"1,000.00",320.42,181.62,1.00,162.75,316.50,481.00,636.00
num_pages,"1,000.00",389.11,229.39,14.00,249.00,352.00,453.00,"2,690.00"
publisher_id,"1,000.00",171.27,99.08,1.00,83.00,177.50,258.00,340.00


Количество явных дуликатов в таблице 0
Размер датасета: (1000, 6)


Итак, `таблица books` состоит из 6 колонок и 1000 записей. Дубликатов и пропущенных значений нет. Все колонки соответствуют корректному типу данных, кроме даты.

In [8]:
basic_func(authors)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


None

,count,mean,std,min,25%,50%,75%,max
author_id,636.00,318.50,183.74,1.00,159.75,318.50,477.25,636.00


Количество явных дуликатов в таблице 0
Размер датасета: (636, 2)


Итак, `таблица authors` состоит из 2 колонки и 636 записей. Дубликатов и пропущенных значений нет. Все колонки соответствуют корректному типу данных.

In [9]:
basic_func(publishers)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


None

,count,mean,std,min,25%,50%,75%,max
publisher_id,340.00,170.50,98.29,1.00,85.75,170.50,255.25,340.00


Количество явных дуликатов в таблице 0
Размер датасета: (340, 2)


Итак, `таблица publishers` состоит из 2 колонок и 340 записей. Дубликатов и пропущенных значений нет. Все колонки соответствуют корректному типу данных.

In [10]:
basic_func(ratings)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


None

,count,mean,std,min,25%,50%,75%,max
rating_id,"6,456.00","3,228.50","1,863.83",1.00,"1,614.75","3,228.50","4,842.25","6,456.00"
book_id,"6,456.00",510.57,284.14,1.00,291.00,506.00,750.00,"1,000.00"
rating,"6,456.00",3.93,0.94,1.00,3.00,4.00,5.00,5.00


Количество явных дуликатов в таблице 0
Размер датасета: (6456, 4)


In [11]:
ratings['book_id'].nunique()

1000

Итак, `таблица ratings` состоит из 4 колонок и 6456 записей. Дубликатов и пропущенных значений нет. Все колонки соответствуют корректному типу данных.

In [12]:
basic_func(reviews)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


None

,count,mean,std,min,25%,50%,75%,max
review_id,"2,793.00","1,397.00",806.41,1.00,699.00,"1,397.00","2,095.00","2,793.00"
book_id,"2,793.00",504.69,288.47,1.00,259.00,505.00,753.00,"1,000.00"


Количество явных дуликатов в таблице 0
Размер датасета: (2793, 4)


In [13]:
reviews['book_id'].nunique()

994

Итак, `таблица reviews` состоит из 4 колонок и 3793 записей. Дубликатов и пропущенных значений нет. Все колонки соответствуют корректному типу данных.

### Расчет количества книг, вышедших после 1 января 2000 года.

In [14]:
query_num_books = ''' SELECT COUNT(DISTINCT book_id)
                      FROM books
                      WHERE (publication_date) > '2000-01-01'
                  '''

In [15]:
num_books = pd.io.sql.read_sql(query_num_books, con = engine) 
num_books

,count
0,819


Количество книг, вышедших после 1 января 2000 года, составляет 819 штук.

### Для каждой книги посчитайте количество обзоров и среднюю оценку

In [16]:
# решение
query_num_text_rating = '''SELECT b.book_id,
                                   b.title,
                                   AVG(r.rating) AS avg_rating,
                                   COUNT(DISTINCT v.review_id) AS text_qty
                            FROM books AS b
                            JOIN ratings AS r ON b.book_id = r.book_id
                            LEFT JOIN reviews AS v ON r.book_id= v.book_id
                            GROUP BY b.book_id, b.title
                            ORDER BY COUNT(v.review_id) DESC   
                        '''

In [17]:
num_text_rating = pd.io.sql.read_sql(query_num_text_rating, con = engine) 
num_text_rating

,book_id,title,avg_rating,text_qty
0,948,Twilight (Twilight #1),3.66,7
1,750,The Hobbit or There and Back Again,4.12,6
2,673,The Catcher in the Rye,3.83,6
3,302,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),4.41,6
4,299,Harry Potter and the Chamber of Secrets (Harry Potter #2),4.29,6
...,...,...,...,...
995,221,Essential Tales and Poems,4.00,0
996,387,Leonardo's Notebooks,4.00,0
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.00,0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.67,0


Количество обзоров text_qty не всегда кореллирует со средним рейтингом avg_rating: иногда обзоров много, а рейтинг низкий, и наоборот: мало обзоров - рейтинг высокий. Срейдний рейтинг книг колеблется от 1.5 до 5. Всего обзору подверглись 994 книги. 6 книг не получили обзора, но имеют оценку.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [18]:
query_publisher_most = '''SELECT p.publisher_id,
                                 p.publisher,
                                 COUNT(b.book_id) AS book_qty
                          FROM publishers AS p
                          JOIN books AS b ON p.publisher_id = b.publisher_id
                          WHERE b.num_pages > 50
                          GROUP BY p.publisher_id, p.publisher
                          ORDER BY COUNT(b.book_id) DESC
                          LIMIT (1);
                       '''

In [19]:
publisher_most = pd.io.sql.read_sql(query_publisher_most, con = engine) 
publisher_most

,publisher_id,publisher,book_qty
0,212,Penguin Books,42


Издательство Penguin Books выпустило 42 книги - наибольшее количество по всему датасету, и это именно книги, а не брошюры!

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [20]:
# решение
query_author_high_rate = '''SELECT Автор,
                                   AVG(СР_оценка) AS Ср_рейтинг
                            FROM (SELECT bs.book_id,
                                   COUNT(DISTINCT rt.rating_id) AS Количество_оценок,
                                   AVG(rt.rating) AS СР_оценка,
                                   ath.author_id,
                                   ath.author AS Автор
                            FROM ratings AS rt
                            JOIN books AS bs ON rt.book_id = bs.book_id
                            JOIN authors AS ath ON bs.author_id=ath.author_id
                            GROUP BY bs.book_id, ath.author_id
                            HAVING COUNT(rt.rating_id) >=50
                            ORDER BY AVG(DISTINCT rt.rating_id) DESC)T
                            GROUP BY Автор
                            LIMIT(1)
                            
                        '''

In [21]:
author_high_rate = pd.io.sql.read_sql(query_author_high_rate, con = engine) 
author_high_rate

,Автор,Ср_рейтинг
0,J.K. Rowling/Mary GrandPré,4.28


Diana Gabaldon - автор с самой высокой средней оценкой среди книг, которые были оценены 50 и более раз.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [24]:
# решение
query_mean_text_50rate = '''SELECT AVG(rev_qty) AS avg_review_qty
                            FROM (SELECT COUNT(rv.review_id) AS rev_qty
                            FROM reviews AS rv
                            WHERE rv.username IN (SELECT rt.username
                            FROM ratings AS rt
                            GROUP BY rt.username
                            HAVING COUNT(rt.rating_id)>50)
                            GROUP BY rv.username) T;
                            
                            '''
mean_text_50rate = pd.io.sql.read_sql(query_mean_text_50rate, con = engine) 
mean_text_50rate


,avg_review_qty
0,24.33


Итак, среднее количество обзоров от пользователей, поставивших более 50ти оценок, составило 24.33.